In [33]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
from nltk.corpus import stopwords 
import re
analyser = SentimentIntensityAnalyzer()

In [34]:
data=pd.read_csv("dress-flipkart-final-final.csv")
data.head()
data=data[['URL','ID','ITEM','STARS','NUMBER OF RATINGS','NUMBER OF REVIEWS', 'LIST OF REVIEWS']]

In [35]:
data.rename(columns = {'ID':'id','ITEM':'desc', 'LIST OF REVIEWS':'reviews','NUMBER OF REVIEWS':'num_reviews','STARS':'stars','NUMBER OF RATINGS':'num_ratings'}, inplace=True)
data.head()

,URL,id,desc,stars,num_ratings,num_reviews,reviews
0,http://flipkart.com/aayu-women-fit-flare-multi...,0,Women Fit and Flare Multicolor Dress,4.0,4510,867,"['nice🙂👍', 'Nice fabric... beautiful dress... ..."
1,http://flipkart.com/micozy-women-a-line-multic...,1,Women A-line Multicolor Dress,4.0,12612,2026,"['very very nice dress 💃💃💃💃💃', 'Good', 'good']"
2,http://flipkart.com/daevish-women-a-line-black...,2,Women A-line Black Dress,4.3,6532,1445,"['I love this dress 👌👌', 'Good quality...', 'P..."
3,http://flipkart.com/limeswood-creation-women-m...,3,"Women Maxi Black, Yellow Dress",3.9,408,50,"['good product', 'Nice dress but finishing was..."
4,http://flipkart.com/tokyo-talkies-women-maxi-b...,4,Women Maxi Black Dress,4.2,1173,283,"['outfit is js osam', 'great dress', ""very ver..."


In [36]:
data.reviews = data.reviews.str.lower()
data.reviews = data.reviews.str.replace('\n','').str.replace('[\'!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]','')

In [37]:
data['vader_score'] = 1.0
for ind in data.index: 
    rev=data['reviews'][ind]
    x = rev.split()
#     print(x)
    sum_score=0.0
    for i in x:
        score = analyser.polarity_scores(i)
        sum_score=sum_score+score['compound']
    data['vader_score'][ind] =sum_score

C:\Users\Miloni Mittal\Anaconda3\ANACONDA\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [38]:
total_star_givers = data['num_ratings'].sum()     
total_review_givers = data['num_reviews'].sum()
# print(total_star_givers)
# print(total_review_givers)

In [39]:
data['final_score'] = ((data['stars']*data['num_ratings'])/total_star_givers)+((data['vader_score']*data['num_reviews'])/total_review_givers)

In [40]:
data.head()

,URL,id,desc,stars,num_ratings,num_reviews,reviews,vader_score,final_score
0,http://flipkart.com/aayu-women-fit-flare-multi...,0,Women Fit and Flare Multicolor Dress,4.0,4510,867,nice🙂👍 nice fabric beautiful dress perfect for...,2.5259,0.040742
1,http://flipkart.com/micozy-women-a-line-multic...,1,Women A-line Multicolor Dress,4.0,12612,2026,very very nice dress 💃💃💃💃💃 good good,1.3023,0.087918
2,http://flipkart.com/daevish-women-a-line-black...,2,Women A-line Black Dress,4.3,6532,1445,i love this dress 👌👌 good quality perfect dress,1.6492,0.055772
3,http://flipkart.com/limeswood-creation-women-m...,3,"Women Maxi Black, Yellow Dress",3.9,408,50,good product nice dress but finishing was not ...,1.1351,0.002576
4,http://flipkart.com/tokyo-talkies-women-maxi-b...,4,Women Maxi Black Dress,4.2,1173,283,outfit is js osam great dress very verry nice...,1.6183,0.010081


In [46]:
data.to_csv('dresses_csv_final.csv')

In [41]:
dict_bigram={}
item_bigram={}
stop_words = set(stopwords.words('english')) 
for ind in data.index:
    d=data['desc'][ind][6:-5]
    d= re.sub(r'[^\w\s]', '', d) 
    nltk_tokens = nltk.word_tokenize(d)
    nltk_tokens = [w for w in nltk_tokens if not w in stop_words] 
    arr=list(nltk.bigrams(nltk_tokens))
    for i in arr:
        if i in dict_bigram:
            dict_bigram[i]=dict_bigram[i]+1
            item_bigram[" ".join(i)].append(ind)
        else:
            dict_bigram[i]=1
            item_bigram[" ".join(i)]=[]
            item_bigram[" ".join(i)].append(ind)
print(item_bigram) 

{'Fit Flare': [0, 5, 6, 9, 11, 17, 34, 37, 40, 43, 67, 77, 80, 81, 83, 84, 87, 98, 102, 103, 107, 110, 114, 122, 123, 131, 135, 139, 142, 143, 144, 147, 148, 156, 158, 172, 176, 180, 181, 184, 193, 195, 198, 201, 205, 219, 222, 244, 245, 248, 254, 265, 267, 276, 277, 284, 286, 287, 289, 290, 297, 301, 304, 309, 316, 320, 322, 323, 325, 331, 333], 'Flare Multicolor': [0, 6, 9, 11, 87, 123, 142, 143, 286, 290, 297], 'Aline Multicolor': [1, 7, 10, 19, 65, 116, 121, 140, 153, 155, 163, 168, 221, 237, 246, 258, 329], 'Aline Black': [2, 27, 55, 62, 125, 128, 129, 164, 218, 250, 260, 279], 'Maxi Black': [3, 4, 175, 206, 216, 226, 231], 'Black Yellow': [3], 'Flare Yellow': [5, 83, 277, 325], 'Yellow Dress': [5, 18, 83], 'Dress With': [5, 18, 23, 83, 84, 87, 114, 265, 290], 'Skater Multicolor': [8, 28, 30, 38, 42, 44, 57, 64, 74, 105, 170, 178, 283, 288], 'Bandage Yellow': [12, 59, 94], 'Gown Maroon': [13, 229], 'Maxi Maroon': [14, 22, 52, 119, 212, 228, 233, 285], 'Maxi Blue': [15, 39, 46, 50,

In [42]:
dict_bigram_sorted= sorted(dict_bigram, key=dict_bigram.get, reverse=True)
print(dict_bigram_sorted)

[('Fit', 'Flare'), ('Dark', 'Blue'), ('Aline', 'Multicolor'), ('Skater', 'Multicolor'), ('Maxi', 'Blue'), ('Maxi', 'Multicolor'), ('Aline', 'Black'), ('Flare', 'Black'), ('Flare', 'Multicolor'), ('Flare', 'Dark'), ('Dress', 'With'), ('Flare', 'Pink'), ('Maxi', 'Maroon'), ('High', 'Low'), ('Sheath', 'Maroon'), ('White', 'Black'), ('Maxi', 'Black'), ('Flare', 'Blue'), ('Maxi', 'White'), ('Maxi', 'Dark'), ('Aline', 'Blue'), ('Shirt', 'Yellow'), ('Maxi', 'Red'), ('Two', 'Piece'), ('Piece', 'Dress'), ('Flare', 'Green'), ('Aline', 'Red'), ('Maxi', 'Pink'), ('Aline', 'White'), ('Flare', 'Yellow'), ('Maxi', 'Yellow'), ('White', 'Pink'), ('Light', 'Blue'), ('Skater', 'Red'), ('Skater', 'Yellow'), ('Flare', 'Maroon'), ('Flare', 'White'), ('Skater', 'Maroon'), ('Yellow', 'Dress'), ('Bandage', 'Yellow'), ('Dark', 'Green'), ('Aline', 'Yellow'), ('Bodycon', 'Maroon'), ('Gown', 'Maroon'), ('Gathered', 'Black'), ('Dress', 'Black'), ('Flare', 'Red'), ('Shirt', 'Blue'), ('Low', 'Blue'), ('Low', 'Red'), 

In [43]:
row=dict_bigram_sorted[0:5]+dict_bigram_sorted[-5:]
print(row)

[('Fit', 'Flare'), ('Dark', 'Blue'), ('Aline', 'Multicolor'), ('Skater', 'Multicolor'), ('Maxi', 'Blue'), ('Sheath', 'Black'), ('Skater', 'White'), ('Green', 'Yellow'), ('Bodycon', 'Dark'), ('Layered', 'Light')]


In [44]:
res = [] 
for el in row: 
#         sub = el.split(', ')
    sub=" ".join(el)
    res.append(list(sub.split("*"))) 
print(res)

[['Fit Flare'], ['Dark Blue'], ['Aline Multicolor'], ['Skater Multicolor'], ['Maxi Blue'], ['Sheath Black'], ['Skater White'], ['Green Yellow'], ['Bodycon Dark'], ['Layered Light']]


In [45]:
import csv
with open('dress_top_bottom.csv', 'w', newline='') as file:
    res = [] 
    for el in row: 
#         sub = el.split(', ')
        sub=" ".join(el)
        res.append(list(sub.split("*"))) 

    for i in range(len(res)):
        res[i]=res[i]+item_bigram[res[i][0]]

    print(res)
    writer = csv.writer(file)
    writer.writerows(res)

[['Fit Flare', 0, 5, 6, 9, 11, 17, 34, 37, 40, 43, 67, 77, 80, 81, 83, 84, 87, 98, 102, 103, 107, 110, 114, 122, 123, 131, 135, 139, 142, 143, 144, 147, 148, 156, 158, 172, 176, 180, 181, 184, 193, 195, 198, 201, 205, 219, 222, 244, 245, 248, 254, 265, 267, 276, 277, 284, 286, 287, 289, 290, 297, 301, 304, 309, 316, 320, 322, 323, 325, 331, 333], ['Dark Blue', 35, 43, 49, 70, 77, 78, 120, 148, 168, 179, 180, 189, 198, 220, 244, 247, 248, 254, 256, 261, 270, 276, 309, 316, 318, 319], ['Aline Multicolor', 1, 7, 10, 19, 65, 116, 121, 140, 153, 155, 163, 168, 221, 237, 246, 258, 329], ['Skater Multicolor', 8, 28, 30, 38, 42, 44, 57, 64, 74, 105, 170, 178, 283, 288], ['Maxi Blue', 15, 39, 46, 50, 137, 177, 197, 230, 300, 303, 306, 312, 314, 327], ['Sheath Black', 313], ['Skater White', 317], ['Green Yellow', 323], ['Bodycon Dark', 326], ['Layered Light', 335]]
